In [77]:
"""
The Autonomous Cooperative Consensus Orbit Determination (ACCORD) framework.
Author: Beth Probert
Email: beth.probert@strath.ac.uk

Copyright (C) 2025 Applied Space Technology Laboratory

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.

"""

import json
import hashlib # TODO - might use a different algorithm? we will see if needed. Start small ffs dont get overwhelmed by the whole ledger
import random
from datetime import datetime, timedelta
import numpy as np
import astropy.constants as ac
from skyfield.api import EarthSatellite, load, wgs84

# Global Variables for consensus
# TODO - May tune these to optimise performance
CONFIRMATION_THRESHOLD = 1
REJECTION_THRESHOLD = -1
CONFIRMATION_STEP = 0.1

# Global astrophysical values
G = ac.G # Gravitational Constant in m3 / (kg s2)
M = ac.M_earth # Mass of the Earth in kg
R = float(ac.R_earth.to("km").value) # Radius of the Earth in km

In [21]:
def load_json_data(file_name: str) -> list:
    """
    Turns json data in a file into a Python dict.
    JSON data must be in the Celestrak format
    https://rhodesmill.org/skyfield/earth-satellites.html
    """
    with load.open(file_name) as f:
        data = json.load(f)

    ts = load.timescale()
    satellite_list = [EarthSatellite.from_omm(ts, fields) for fields in data]
    return satellite_list

# Retrieve data
od_data = load_json_data("od_data.json")

In [22]:
class Transaction():
    """
    Transaction containing information to be submitted in the Distributed Ledger.
    Consensus needs to be reached on the validity of the information received.
    """
    def __init__(self, sender_address, recipient_address, sender_private_key, timestamp, tx_data, consensus_reached: bool = False, is_confirmed: bool = False) -> None:
        self.sender_address: int = sender_address
        self.recipient_address: int = recipient_address
        self.sender_private_key: str = sender_private_key
        self.timestamp: datetime = timestamp
        self.consensus_reached: bool = consensus_reached
        self.is_confirmed: bool = is_confirmed
        # TODO Add cooperative OD information to TLE data json
        # tx_data will include cooperative OD data, so including the ID 
        # and trajectory info for a witnessed satellite as well.
        self.tx_data: str = tx_data
        
        self.parent_hashes: list[str] = []
        self.confirmation_score: float = 0
        self.is_rejected: bool = False

        self.hash = self.calculate_hash()
    
    def __repr__(self) -> str:
        return (f"Transaction(\n"
                f"  sender_address={self.sender_address},\n"
                f"  recipient_address={self.recipient_address},\n"
                f"  timestamp={self.timestamp},\n"
                f"  tx_data='{self.tx_data}',\n"
                f"  consensus_reached={self.consensus_reached},\n"
                f"  is_confirmed={self.is_confirmed},\n"
                f"  is_rejected={self.is_rejected},\n"
                f"  confirmation_score={self.confirmation_score},\n"
                f"  parent_hashes={self.parent_hashes},\n"
                f"  hash={self.hash[:10]}...)\n")
        
    def calculate_hash(self) -> str:
        """
        Calculates a hash string for a transaction. The hash is generated by encoding all of
        the transaction's unique information, so that ANY change to a transaction results in a new 
        hash, making tampering easier to identify.
        """
        return hashlib.sha256(str(self.sender_address).encode() + str(self.timestamp).encode() + 
                              self.sender_private_key.encode() + self.tx_data.encode() + 
                              str(self.recipient_address).encode()
                             ).hexdigest()

In [23]:
class SatelliteNode():
    """
    A class representing a node in the network, in this case a LEO satellite. 
    This does NOT represent a node in the ledger - these are transactions
    """
    def __init__(self, id: str) -> None:
        self.id: str = id
        # Reputation starts at 0, affected by validity and accuracy
        self.reputation: float = 0 #TODO - need to consider how this affects consensus. If reputation low, does it get allowed? or does it affect consensus score?

In [24]:
class DAG():
    """
    A class representing the Directed Acyclic Graph (DAG) Distributed Ledger Technology.
    When a transaction is received, it is added to the DAG. The number of parents
    for each transaction is decided using a tip selection algorithm.
    """
    
    def __init__(self) -> None:
        # TODO - need a way to check that the DAG has not been tampered with

        # Ledger structure is:
        # key: string hash of transaction, value: Transaction class
        self.ledger: dict = self.create_genesis_tx()
    
    def create_genesis_tx(self) -> dict[str, list[Transaction]]:
        """
        Creates the two genesis transactions to initialise the DAG and provide parents
        for the first real transaction.
        Have to set consensus_reached and is_confirmed = True here otherwise strong parents become impossible.
        """
        return {"Genesis Transaction 1": [Transaction(0, 0, "1234", datetime.now(), "Genesis Transaction 1", consensus_reached=True, is_confirmed=True)],
                "Genesis Transaction 2": [Transaction(0, 0, "5678", datetime.now(), "Genesis Transaction 2", consensus_reached=True, is_confirmed=True)]}
    
    def check_thresholds(self, transaction: Transaction) -> None:
        """
        Check if the transaction confirmation or rejection thresholds have been crossed. 
        This will affect weighting. is_confirmed = strong weighting, else weak weighting
        """
        if REJECTION_THRESHOLD <= transaction.confirmation_score <= CONFIRMATION_THRESHOLD:
            transaction.confirmation_score +=  CONFIRMATION_STEP
        else:
            transaction.confirmation_score -= CONFIRMATION_STEP
        
        if transaction.confirmation_score >= CONFIRMATION_THRESHOLD:
            transaction.is_confirmed = True
        elif transaction.confirmation_score <= REJECTION_THRESHOLD:
            transaction.is_rejected = True
    
    def get_parents(self) -> tuple[str, str]:
        """
        Randomly select 2 parents for the transaction
        # THRESHOLDS affect tip selection for parents - TODO
        """
        return tuple(random.sample(list(self.ledger.keys()), 2))
    
    def add_tx(self, transaction: Transaction) -> None:
        """
        Add a transaction to the DAG
        """
        # TODO - either this or the epoch of the TLE
        transaction.timestamp = datetime.now()

        # TODO - tx or blocks?? start with tx for now
        # TODO - fixed number of parents: 2
        parent1, parent2 = self.get_parents()
        
        # There is guaranteed to be two parent - the genesis transactions in the DAG.
        transaction.parent_hashes.extend([parent1, parent2])

        # Add transaction to ledger
        self.ledger[transaction.calculate_hash()] = [transaction]
        # TODO - need to add consensus mechanism in here, may need to be a function within this class rather than a separate class to avoid circles
        self.check_thresholds(transaction)

# Test code
test_dag = DAG()
test_tx = Transaction(0, 0, "1111", datetime.now(), "Test Transaction")
test_dag.add_tx(test_tx)
print(test_dag.ledger[test_tx.calculate_hash()])

[Transaction(
  sender_address=0,
  recipient_address=0,
  timestamp=2025-07-28 14:38:38.865480,
  tx_data='Test Transaction',
  consensus_reached=False,
  is_confirmed=False,
  is_rejected=False,
  confirmation_score=0.1,
  parent_hashes=['Genesis Transaction 2', 'Genesis Transaction 1'],
  hash=2f82f0e992...)
]


In [78]:
class ConsensusMechanism():
    """
    The Proof of Inter-Satellite Evaluation (PoISE) consensus mechanism.
        
    TODO - need to check: if physically possible, how many times its been seen (maybe new param?)
    """
    # TODO - proof of Location XYO paper, and bidirectional heuristics (need to check for invalid, or valid but incorrect/inaccurate)(4.3, proof of proximity)
    # TODO - format for data? class for verification/consensus? Need some calcs
    # TODO TODAY - SLAM DRONE PAPER, and consensus on position, doppler shift?? what data so I have to choose from?
    # DAGmap - map consensus could be something to build upon? does it have a ground truth?
    # PowerGraph - consensus for trust level, calculates validity of transacion from probability level. I guess I need to calculate validity from some maths? possible - if yes, hen how accurate/likely
    # probability distruibution for observations? Algorithm one in PowerGraph paper
    
    def __init__(self):
        self.consensus_threshold : float = 0.6 # TODO - tune
        self.reputation_step: float = 0.1 # TODO - tune
    
    def data_is_valid(self, sat: EarthSatellite) -> bool:
        """
        Check if received data is valid, i.e. physically and logically possible for a LEO satellite
        Assume data comes in a standard TLE format from a Celestrak JSON
        Checks validity to reduce computation effort in consensus for data that is impossible
        Assumes that TLE data is received and transformed correctly
        TODO - find out what witness location data will look like for comparison and consensus
        To prevent all of SpaceX agreeing that they didn't see any satellites over Ukraine, nope, no way, they were in geostationary orbit above the North Pole 
        TODO - Josh idea, think about seeding the initial conditions with ground station data until I have built up enough of a map. Could also be GNSS/GPS data if that works. Dont really mind           
        """
        # Use sgp4 propogation to get altitude and velocity
        # More suitable than keplerian motion
        # The Two-Body-Keplerian orbit propagator is the less precise because 
        # it simplifies the gravitational field of the Earth as spherical and 
        # ignores all other sources of orbital perturbations. The SGP4 
        # orbit propagator enhances the accuracy by considering the 
        # oblateness of the Earth and the effects of atmospheric drag.
        epoch = sat.epoch
        state_vector = sat.at(epoch) # works in ITRF frame
        altitude_km = (wgs84.height_of(state_vector)).km
        velocity = state_vector.velocity.km_per_s
        speed_kmps = np.linalg.norm(velocity)

        # Circular orbit: Min speed 6.9 km/s
        # Elliptical orbit: Max speed 10.07 km/s
        # Plus buffer for error
        if not (6.5 <= speed_kmps <= 10.5):
            return False

        # For LEO, satellites should have an inclination between 0 and 180 degrees
        # Inclination is initially provided in radians
        if not (0 <= (sat.model.inclo * 180 / np.pi) <= 180):
            return False
        
        # See https://rhodesmill.org/skyfield/earth-satellites.html#detecting-propagation-errors
        # If any elements don't make sense, the position is returned as [nan, nan, nan]
        # e.g. if eccentricity is not between 0 and 1
        if np.nan in state_vector.xyz.km:
            return False
        
        # Altitude should be in LEO range of 200km to 2000km above the Earth's surface
        if not (200 <= altitude_km <= 2000):
            return False

        return True
    
    def get_correctness_score(self, sat: EarthSatellite, dag: DAG) -> float:
        """
        Determine the correctness score for transactional data being added to the DAG.
        This is done by comparing the data provided by a satellite with historical data in the DAG.
        TODO - change
        Returns a correctness score between 0 (inconsistent) and 1 (high agreement)

        """
        # Try to match OBJECT_ID in historical transactions (basic implementation)

        matches = []

        for tx_list in dag.ledger.values():
            for tx in tx_list:
                try:
                    past_data = json.loads(tx.tx_data)
                    if isinstance(past_data, dict) and past_data.get("OBJECT_ID") == sat.model.satnumid:
                        matches.append(past_data)
                except Exception:
                    continue
        
        if not matches:
            return 0.5 # Neutral if not seen before
        
        # Very basic check — mean motion deviation, inclination deviation
        past = matches[-1]
        motion_dev = abs(sat.model.no_kozai - float(past["MEAN_MOTION"]))
        incl_dev = abs((sat.model.inclo * 180 / np.pi) - float(past["INCLINATION"]))

        correctness_score = 1.0
        if motion_dev > 0.1:
            correctness_score -= 0.3 # TODO - tune
        if incl_dev > 5:
            correctness_score -= 0.3 # TODO - tune
        return max(0.0, min(1.0, correctness_score))
    
    def estimate_accuracy(self, sat: EarthSatellite) -> float:
        """
        Rough placeholder accuracy metric.
        TODO: Replace with actual noise/uncertainty model.
        """
        # Use a dummy normal distribution around ideal speed 7.8 km/s
        ideal_speed = 7.8
        velocity = sat.at(sat.epoch).velocity.km_per_s
        speed = np.linalg.norm(velocity)

        deviation = abs(speed - ideal_speed)
        if deviation < 0.3:
            return 1.0
        elif deviation < 0.6:
            return 0.7
        elif deviation < 1.0:
            return 0.5
        else:
            return 0.2

    def calculate_consensus_score(self, correctness: float, accuracy: float, reputation: float) -> float:
        """
        Weighted score. Tuneable weights.
        """
        return 0.4 * correctness + 0.4 * accuracy + 0.2 * reputation                
    
    def proof_of_inter_satellite_evaluation(self, dag: DAG, sat_node: SatelliteNode, transaction: Transaction) -> bool:
        """
        Returns a bool of it consensus has been reached  
        NOTE: Assume one witnessed satellite per transaction   
        """
        # 1)  Get the data 
        od_data = load_json_data("od_data.json")
        
        # 2) TODO Check if data is valid, if not - ignore. Consensus cannot be reached on invalid data. If yes, add to DAG
        # If the list is empty, there is no data that can be valid
        if len(od_data) == 0:
            # Reduce node reputation for providing no data that can be checked
            sat_node.reputation -= self.reputation_step
            return False
        
        for sat in od_data:
            if not self.data_is_valid(sat):
                # Reduce node reputation for providing invalid data
                sat_node.reputation -= self.reputation_step
                return False

        # 3a) If we have enough, valid data, submit a transaction with the string of data
        # TODO - might be able to optimise this to reduce string length
        # Serialize data for transaction
        tx_data_dict = {
        "OBJECT_NAME": sat.name,
        "OBJECT_ID": sat.model.satnum,
        "EPOCH": str(sat.epoch.utc_iso()),
        "MEAN_MOTION": (sat.model.no_kozai / (2 * np.pi)) * 1440, # Convert rads/min into revs/day
        "ECCENTRICITY": sat.model.ecco,
        "INCLINATION": sat.model.inclo * 180 / np.pi
        }

        transaction.tx_data = json.dumps(tx_data_dict)
    
        dag.add_tx(transaction)
        
        # 3) TODO Check we have enough data to be bft (3f + 1)
        # if not, consensus cannot be reached. 
        # Length of DAG is 2 by default with genesis transactions. These are dummy data,
        # so we must have at least 4 real transactions on top to be BFT (2 + 4 =  total)
        if len(dag.ledger) < 6:
            return False
        
        # 4) TODO Check if satellite has been witnessed before
        #4a if yes, does this data agree with other data/ is it correct? Assign correctness score -> affects transaction
        #  This is going to be very tricky. How do I get this data?? Where do I store it? Do I want this to tie in to how parents are selected?
        correctness_score = self.get_correctness_score(tx_data_dict, dag)
        
        # 5) TODO is sensor data accurate (done regardless of previous witnessing). Assign accuracy score -> affects transaction and node reputation
        # Again, might be tricky. Probability distribution here? Like in the PowerGraph paper?
        accuracy_score = self.estimate_accuracy(sat)
        
        # 6) TODO calculate consensus score - node reputation, accuracy and correctness all factor
        # Need to develop an equation - this will take some reading and tuning

        consensus_score = self.calculate_consensus_score(correctness_score, accuracy_score, sat_node.reputation)

        # if consensus reached - strong node (maybe affects node reputation?), else weak node (like IOTA)
        if consensus_score >= self.consensus_threshold:
            transaction.consensus_reached = True
            sat_node.reputation += self.reputation_step
            return True
        else:
            transaction.consensus_reached = False
            sat_node.reputation -= self.reputation_step
            return False
        
        # 7) TODO if consensus score above threshold, consensus reached. Else not.
     
        # if all history is strong - strong edge connection, else weak edge connection. I guess I can't update edge connections or tx strength retroactively so..
        # this bit is the same as tangle, so i may just try and run this consensus on a tangle like system or simulator like in references 

        # TODO - steps 4-6 are the key bits, work here. Get equations and find some tuning evidence. Need to transform into a DAG from a blockchain after that

In [ ]:
# Tester demo function
def run_consensus_test():
    # Number of transactions must be >= 3 for consensus to be possible
    n_tx = 3
    print("=== Running Consensus Test ===")

    # Load observation data from your JSON file
    satellites = load_json_data("od_data.json")
    if not satellites:
        print("No satellite data found.")
        return

    # Setup
    test_satellite = SatelliteNode(id="SAT-001")
    test_dag = DAG()
    base_sat = satellites[0]

    # Add other transactions with some random noise
    # TODo - test with bigger variants
    for i in range(n_tx):
        # Simulate noise in observation
        noise = {
            "mean_motion": np.random.normal(0, 0.01),       # rev/day
            "eccentricity": np.random.normal(0, 0.00002),    # unitless
            "inclination": np.random.normal(0, 0.05),        # degrees
            "epoch_jitter": np.random.normal(0, 2)           # seconds
        }

        observed_epoch = base_sat.epoch.utc_datetime() + timedelta(seconds=noise["epoch_jitter"])

        dummy_tx = Transaction(
            sender_address=f"NODE-{i}",
            recipient_address="GROUND",
            sender_private_key="dummy_key",
            timestamp=datetime.now(),
            tx_data=json.dumps({
                "OBJECT_NAME": base_sat.name,
                "OBJECT_ID": 25544,
                "EPOCH": observed_epoch.isoformat() + "Z",
                "MEAN_MOTION": ((base_sat.model.no_kozai / (2 * np.pi)) * 1440) + noise["mean_motion"],
                "ECCENTRICITY": base_sat.model.ecco + noise["eccentricity"],
                "INCLINATION": (base_sat.model.inclo * 180 / np.pi) + noise["inclination"]
            })
        )
        test_dag.add_tx(dummy_tx)

    test_transaction = Transaction(sender_address=test_satellite.id,
                                   recipient_address="GROUND",
                                   sender_private_key="dummy_key",
                                   timestamp=datetime.now(),
                                   tx_data="")

    # Init PoISE mechanism
    poise = ConsensusMechanism()

    # Run consensus on a single satellite observation
    consensus_result = poise.proof_of_inter_satellite_evaluation(
        dag=test_dag,
        sat_node=test_satellite,
        transaction=test_transaction
    )

    # Output result
    print(f"\nConsensus reached: {consensus_result}")
    print(f"Updated Node Reputation: {test_satellite.reputation:.2f}")
    print(f"Transaction:\n{test_transaction}")
    last_index = list(test_dag.ledger)[-1]
    penultimate_index = list(test_dag.ledger)[-2]
    print("Final Tx:")
    print(test_dag.ledger[last_index])
    print("Penultimate Tx:")
    print(test_dag.ledger[penultimate_index])

run_consensus_test()

=== Running Consensus Test ===

Consensus reached: True
Updated Node Reputation: 0.10
Transaction:
Transaction(
  sender_address=SAT-001,
  recipient_address=GROUND,
  timestamp=2025-07-28 17:34:13.968365,
  tx_data='{"OBJECT_NAME": "ISS (ZARYA)", "OBJECT_ID": 25544, "EPOCH": "2024-05-06T19:53:05Z", "MEAN_MOTION": 15.509576740000002, "ECCENTRICITY": 0.000358, "INCLINATION": 51.639300000000006}',
  consensus_reached=True,
  is_confirmed=False,
  is_rejected=False,
  confirmation_score=0.1,
  parent_hashes=['6f62136e20c8c475c1c6478579d179204450a55bca4a219a636860209570419c', 'Genesis Transaction 2'],
  hash=53d3302c73...)

Final Tx:
[Transaction(
  sender_address=SAT-001,
  recipient_address=GROUND,
  timestamp=2025-07-28 17:34:13.968365,
  tx_data='{"OBJECT_NAME": "ISS (ZARYA)", "OBJECT_ID": 25544, "EPOCH": "2024-05-06T19:53:05Z", "MEAN_MOTION": 15.509576740000002, "ECCENTRICITY": 0.000358, "INCLINATION": 51.639300000000006}',
  consensus_reached=True,
  is_confirmed=False,
  is_rejected